In [45]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [49]:
# Store insurance csv into dataframe
csv_file="../Insurance-Challange/Resources/Dataset.csv"
insurance_data_df=pd.read_csv(csv_file)
insurance_data_df.head()

C:\Users\mraza\Python\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2,3,4,5,6,7,8,10,15,16,17,19,20,23,24,31,32,33,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,No,Policy Number,Name of Insured,CARRIER,Premium written,Premium earned,Premium unearned,Effective date,Expiration date,Cancelations,...,Processing fee,Installment Fe,Policy Fee,Unnamed: 30,SYSTEM,PLCYPREM_W,PLCYPREM_E,Unnamed: 34,Tax reference,Commission basis
0,1.0,CA10000204703,Superior Stone Distributors Inc,BIC,-,66.28,-,1-Jun-18,1-Jun-19,NaN,...,0.0,0.0,0.0,NaN,POLICY DECISIONS,-,66.28,CA10000204703Superior Stone Distributors Inc,GA-COMMERCIAL AUTOMOBILE,WRITTEN
1,2.0,CA10000204703,Superior Stone Distributors Inc,BIC,-,"5,979.55",-,1-Jun-18,1-Jun-19,NaN,...,0.0,0.0,0.0,NaN,POLICY DECISIONS,0,"5,979.55",CA10000204703Superior Stone Distributors Inc,GA-COMMERCIAL AUTOMOBILE,WRITTEN
2,3.0,CA10000204703,Superior Stone Distributors Inc,BIC,-,"3,154.39",-,1-Jun-18,1-Jun-19,NaN,...,0.0,0.0,0.0,NaN,POLICY DECISIONS,-,"3,154.39",CA10000204703Superior Stone Distributors Inc,GA-COMMERCIAL AUTOMOBILE,WRITTEN
3,4.0,CA10000204704,Superior Stone Distributors Inc,BIC,"18,712.00","6,237.33","12,474.67",1-Jun-19,1-Jun-20,NaN,...,0.0,0.0,0.0,NaN,POLICY DECISIONS,"18,712.00","6,237.33",CA10000204704Superior Stone Distributors Inc,GA-COMMERCIAL AUTOMOBILE,WRITTEN
4,5.0,CA10000204704,Superior Stone Distributors Inc,BIC,"5,708.00","1,902.67","3,805.33",1-Jun-19,1-Jun-20,NaN,...,0.0,0.0,0.0,NaN,POLICY DECISIONS,"5,708.00","1,902.67",CA10000204704Superior Stone Distributors Inc,GA-COMMERCIAL AUTOMOBILE,WRITTEN


In [54]:
#Clean insurance data frame by importing only the relevant columns
new_insurance_data_df=insurance_data_df[['Policy Number', 
                                         'State of Coverage', 
                                         'Line of Business', 
                                         'Premium written',
                                         'Premium earned',
                                         'Premium unearned',
                                         'Audit Premium',
                                         'Effective date',
                                         'Expiration date',
                                         'COMMISSION_RATE',
                                         'Renewal/New business',
                                         'Agent Name',
                                         'State Tax Rate',
                                         'Commission basis',
                                         'Reinsurance premium rate']].copy()
new_insurance_data_df.head()

,Policy Number,State of Coverage,Line of Business,Premium written,Premium earned,Premium unearned,Audit Premium,Effective date,Expiration date,COMMISSION_RATE,Renewal/New business,Agent Name,State Tax Rate,Commission basis,Reinsurance premium rate
0,CA10000204703,GA,COMMERCIAL AUTOMOBILE,-,66.28,-,100.0,1-Jun-18,1-Jun-19,0.15,R,ACRISURE LLC,3.6,WRITTEN,80%
1,CA10000204703,GA,COMMERCIAL AUTOMOBILE,-,"5,979.55",-,100.0,1-Jun-18,1-Jun-19,0.15,R,ACRISURE LLC,3.6,WRITTEN,80%
2,CA10000204703,GA,COMMERCIAL AUTOMOBILE,-,"3,154.39",-,100.0,1-Jun-18,1-Jun-19,0.15,R,ACRISURE LLC,3.6,WRITTEN,80%
3,CA10000204704,GA,COMMERCIAL AUTOMOBILE,"18,712.00","6,237.33","12,474.67",0.0,1-Jun-19,1-Jun-20,0.15,R,ACRISURE LLC,3.6,WRITTEN,80%
4,CA10000204704,GA,COMMERCIAL AUTOMOBILE,"5,708.00","1,902.67","3,805.33",0.0,1-Jun-19,1-Jun-20,0.15,R,ACRISURE LLC,3.6,WRITTEN,80%


In [55]:
# rename columns
clean_insurance_data_df = new_insurance_data_df.rename(columns={"Policy Number": "Policy Number", 
                                         "State": "State of Coverage", 
                                         "Line of Business":"Line of Business", 
                                         "Premium Written":"Premium written",
                                         "Premium Earned":"Premium earned",
                                         "Premium Unearned":"Premium unearned",
                                         "Audit Premium":"Audit Premium",
                                         "Effective Date":"Effective date",
                                         "Expiration Date":"Expiration date",
                                         "Commission Rate":"COMMISSION_RATE",
                                         "New or Renewal":"Renewal/New business",
                                         "Agent":"Agent Name",
                                         "State Tax Rate":"State Tax Rate",
                                         "Commission Basis":"Commission basis",
                                         "Reinsurance premium rate":"Reinsurance premium rate"})
new_insurance_data_df.head()

,Policy Number,State of Coverage,Line of Business,Premium written,Premium earned,Premium unearned,Audit Premium,Effective date,Expiration date,COMMISSION_RATE,Renewal/New business,Agent Name,State Tax Rate,Commission basis,Reinsurance premium rate
0,CA10000204703,GA,COMMERCIAL AUTOMOBILE,-,66.28,-,100.0,1-Jun-18,1-Jun-19,0.15,R,ACRISURE LLC,3.6,WRITTEN,80%
1,CA10000204703,GA,COMMERCIAL AUTOMOBILE,-,"5,979.55",-,100.0,1-Jun-18,1-Jun-19,0.15,R,ACRISURE LLC,3.6,WRITTEN,80%
2,CA10000204703,GA,COMMERCIAL AUTOMOBILE,-,"3,154.39",-,100.0,1-Jun-18,1-Jun-19,0.15,R,ACRISURE LLC,3.6,WRITTEN,80%
3,CA10000204704,GA,COMMERCIAL AUTOMOBILE,"18,712.00","6,237.33","12,474.67",0.0,1-Jun-19,1-Jun-20,0.15,R,ACRISURE LLC,3.6,WRITTEN,80%
4,CA10000204704,GA,COMMERCIAL AUTOMOBILE,"5,708.00","1,902.67","3,805.33",0.0,1-Jun-19,1-Jun-20,0.15,R,ACRISURE LLC,3.6,WRITTEN,80%


In [59]:
#import state and local sales tax csv
csv_file_two="../Insurance-Challange/Resources/State-and-Local-Sales-Tax-Rates-2020.csv"
tax_data_df=pd.read_csv(csv_file_two)
tax_data_df

,State,State Tax Rate
0,Ala.,4.00%
1,Alaska,0.00%
2,Ariz.,5.60%
3,Ark.,6.50%
4,Calif. (b),7.25%
5,Colo.,2.90%
6,Conn.,6.35%
7,D.C.,6.00%
8,Del.,0.00%
9,Fla.,6.00%
